## EDA
Basic EDA to understand the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

### 0. Root repo

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

### 1. Load data

In [ ]:
path_data = 'artifacts/data/data_raw.pkl'
data = pd.read_pickle(path_data)
data.head()

### 2. EDA Basic 1

#### 2.1 null values

In [ ]:
data.isnull().sum()

#### 2.2 descriptive statistics

In [ ]:
data.describe()

### 2.3 Unique Values

In [ ]:
list_features_cat = ['region', 'year', 'month', 'peak']
for feature_cat in list_features_cat:
    print('FEATURE: ', feature_cat)
    print('number of unique regions: ', data[feature_cat].nunique())
    print('unique regions: ', data[feature_cat].unique())
    print('\n')

#### 2.4 Histograms

In [ ]:
def plot_hist(df, features_to_plot):
    """
    Dado un histograma y un listado de features, plotear histograma de cada una de las features
    Args:
        df: dataframe
        features_to_plot: list - features to plot
    """

    # shape sub plots
    number_columns = 1 #fixed
    number_rows = len(features_to_plot)
    
    # create plot
    fig = make_subplots(rows = number_rows, cols = number_columns)
    
    
    # append subplots
    for index_feature in range(number_rows):
        
        # plot
        fig.append_trace(
            #px.histogram(df, x = features_to_plot[index_feature]),
            go.Histogram(x=df[features_to_plot[index_feature]]),
            row=index_feature + 1, 
            col = 1
        )
        
        #Update x-axis title for each subplot
        fig.update_xaxes(title_text=features_to_plot[index_feature], row=index_feature + 1, col=1)
    
    
    fig.update_layout(height=1600, width=600, title_text="Histograms")
    fig.show()

In [ ]:
plot_hist(df = data,
          features_to_plot = data.columns.tolist()[1:]
         )

### 3. Observe trend in data - group data for all US data. No considering the differents regions

#### 3.1 Group data

In [ ]:
df_Total_US = data[data["region"] == "Total_US"]
df_Total_US.head()

#### 3.2 Sales over the years

In [ ]:
### sales over the years

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
mean = df_Total_US.groupby("year")["units_sold"].mean()
std = df_Total_US.groupby("year")["units_sold"].std()
axes.errorbar(mean.index, mean, xerr=0.5, yerr=2 * std, linestyle="")
axes.set_ylabel("Units Sold (millions)")
axes.set_xlabel("Year")

fig.tight_layout()

#### 3.3 Trend Sales Over year

In [ ]:
### time series sales over the years

# sort data
df_timeseries = df_Total_US.sort_values(by = ['date'], ascending = True)

# plot timeseries
fig = px.line(df_timeseries, x='date', y="units_sold")
fig.show()

#### 3.4 Seasonality

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

mean = df_Total_US.groupby("month")["units_sold"].mean()
std = df_Total_US.groupby("month")["units_sold"].std()

axes.errorbar(mean.index, mean, xerr=0.5, yerr=2 * std, linestyle="")
axes.set_ylabel("Units Sold (millions)")
axes.set_xlabel("Month")

fig.tight_layout()

plt.xlabel("Month")
axes.set_xticks(range(1, 13))
plt.ylabel("Units sold (millions)")
plt.show()

#### 3.5 Correlations

In [ ]:
# calculate correlations
columns_to_corr = ["units_sold", "price", "year", "peak"]
corr_df = df_Total_US[columns_to_corr].corr()
corr_df = corr_df.round(2)

# plot correlations
fig = px.imshow(corr_df, width=1000, height=500, text_auto=True, labels = dict(x='Correlations of all features'))
fig.update_xaxes(side="top")
fig.show()

### 4. Analysis by regions

#### 4.1 Delete the region Total US

In [ ]:
# create a dataframe with all regions, deleting the total_US
regions = [
    "Great_Lakes",
    "Midsouth",
    "Northeast",
    "Northern_New_England",
    "SouthCentral",
    "Southeast",
    "West",
    "Plains",
]
data = data[data.region.isin(regions)]

In [ ]:
data.head()

In [ ]:
data.shape

#### 4.2 Solds each region

In [ ]:
# plot units sold of each region


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

mean = data.groupby("region")["units_sold"].mean()
std = data.groupby("region")["units_sold"].std()

axes.errorbar(range(len(mean)), mean, xerr=0.5, yerr=2 * std, linestyle="")

fig.tight_layout()

plt.xlabel("Region")
plt.xticks(range(len(mean)), pd.DataFrame(mean)["units_sold"].index, rotation=20)
plt.ylabel("Units sold (millions)")
plt.show()

### 4.3 Scatter plot - prices vs solds - colored by region - graph p vs q

In [ ]:
plt.figure(figsize=(10, 6))
r_plt = sns.scatterplot(data = data,
                        y = 'price',
                        x = 'units_sold', 
                        hue = 'region'
                       )
r_plt.legend(fontsize=8)
plt.show()

### 5. Comparison between regions. Relation of prices and solds considering the efect of other regions

#### 5.1 generate data

In [ ]:
# filter usefull columns
list_columns_to_comparation = ['date', 'units_sold', 'price', 'region', 'peak']
df_filtered_columns = data[list_columns_to_comparation]
df_filtered_columns

In [ ]:
#### Pivot data to generate a pivot table with the prices for each region in one row
df_pivot_prices = df_filtered_columns.pivot(index='date', columns='region', values='price').reset_index()
df_pivot_prices.columns = ['date'] + ['price_' + col.lower().replace(' ', '_') for col in df_pivot_prices.columns[1:]]
df_pivot_prices

In [ ]:
# delete original column price, with the price in this region
df_filtered_columns = df_filtered_columns.drop(columns = ['price']) 
df_filtered_columns.head()

In [ ]:
#### merge the original data with units solds and region with the data that have the price for each region
df_prices_regions = pd.merge(df_filtered_columns, df_pivot_prices, on='date', how='left')
df_prices_regions.head(9)

In [ ]:
##### COMPARAR DATAFRAMES ESTEN BIEN
data.head(8)

#### 5.2 Correlation between prices

In [ ]:
df_aux_corr_prices = df_prices_regions.drop(columns = ['date', 'units_sold', 'region', 'peak'])
corr_prices = df_aux_corr_prices.corr()
corr_prices = corr_prices.round(3)

# plot
fig = px.imshow(corr_prices, width=1300, height=800, text_auto=True, labels = dict(x=f'Correlations prices each region'))
fig.update_xaxes(side="top")
fig.show()

#### 5.3 Correlation Units sols in region X considering the price of all region

In [ ]:
#### MAKE CORRELATION OF THE UNITS SOLD IN EACH REGION WITH THE PRICES OF EACH REGION
for region_name in df_prices_regions['region'].unique().tolist():
    print(region_name)

    # filter by region
    df_aux_corr = df_prices_regions[df_prices_regions['region'] == region_name]
    df_aux_corr = df_aux_corr.drop(columns = ['date', 'region', 'peak']) # delete column to not show in correlation heatmap
    
    # correlation target
    corr = df_aux_corr.corr()
    corr_units_sold = corr[['units_sold']]
    corr_units_sold =  corr_units_sold.T
    corr_units_sold = corr_units_sold.round(3)
    
    # plot
    fig = px.imshow(corr_units_sold, width=1000, height=400, text_auto=True, labels = dict(x=f'Correlations units solds of region: {region_name}'))
    fig.update_xaxes(side="top")
    fig.show()

### Insights:
- Es la misma cantidad de datos por cada región. Hay un total de 6 regiones (6 regiones de US)
- Existe una cierta variación en la cantidad de ventas de cada año, pero sin una tendencia clara a la alza o baja (ventas a nivel total US)
- A nivel de agregación mensual, se observa meses con mayores ventas que otros, seasonnality, que también se puede ver en el gráfico de las tendencias
- Todas las regiones tienen precios altamente correlacionados, algunas más que otras pero todas con correlaciones de precios sobre 0.6 y los más altos con correlaciones de 0.8
- **Se ven que en algunos sectores/regiones el precio es más elástico al precio que otros**